---

# The Single Neuron Model: The Perceptron & Predicting Next-Day Price Direction

The perceptron is a machine learning technique that is based on the behaviour of biological neurons. Broadly speaking, a perceptron has three components: 
- Pre-activation---a linear combination of weights plus a bias that serves as the "stimulus" for the neuron
- Post-activation---if the stimulus is "intense" enough, then the neuron "fires" and returns a value according to some rule
- Activation function---once the neuron "fires," it processes the stimulus through a predefined function, $\phi$

I will train and implement this model on Quandl’s “WIKI EOD Stock Prices” dataset to demonstrate how it works and some real-world applications. Let's begin by importing the relevant libraries.

---

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from perceptron import *

---

Next, load in the dataset that you'll be training on. As stated above, I'll be training the model on Quandl's "WIKI EOD Stock Prices" dataset. Note my unique API key in the code below; you can get your own with a free account.

---

In [1]:
import quandl
quandl.ApiConfig.api_key = 'oSxcqfNamEjr4aFAmzfD' # My API key
df = quandl.get("WIKI/AAPL", start_date="2010-01-01", end_date="2018-01-01")

QuandlError: (Status 403) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@quandl.com.